# Optimization Problem
In this tutorial, we will learn how to solve the following optimization problem using Gurobi solver.
\begin{align*}
& \max && x + y + z \\
& \textrm{subject to} && x + y = 1 \\
&&& x + 5 y + 2 z \le 10 \\
&&& 0 \le x \le 5 \\
&&& z \ge 2
\end{align*}

There are three components of above optimization problem:
### Decisions: $x, y, z$
### Objective function: $x + y + z$
### Constraints: $x + y = 1, x + 5 y + 2 z \le 10, 0 \le x \le 5, z \ge 2$


#### Write the above optimization problem as below in a notepad file and save it as 'model.lp'

```
Maximize
  x + y + z
Subject To
  c0: x + y = 1
  c1: x + 5 y + 2 z <= 10
Bounds
  0 <= x <= 5
  z >= 2
Generals
  x y z
End
```

To solve this model, we start a python session. We begin by importing Gurobi python package. 

In [30]:
import gurobipy as gp # import gurobipy package as gp
from gurobipy import GRB # Import class GRoB from gurobi package.
import sys

Next step is to read the model. ```gp.read()``` reads the model from file. The argument of this function should be the location of ```.lp``` file. Note that the location uses forward slashes ```/``` not backward slashes ```\```. We assign the model read from this file to a variable named ```model```.

In [32]:
model = gp.read('model.lp')

Read LP format model from file model.lp
Reading time = 0.00 seconds
: 2 rows, 3 columns, 5 nonzeros


To switch off the verbose, we set ```model.Params.outputFlag``` to 0. If we want to see the details of the solution method, we set it equal to 1. Now it is time to solve the model. ```model.optimize()``` optimizes the model. 

In [33]:
model.Params.outputFlag = 0
model.optimize()

After we optimize, we need to find out whether the model is:

* Feasible and Gurobi has found the optimal solution ```GRB.OPTIMAL```
* Infeasible ```GRB.INFEASIBLE```
* Unbounded ```GRB.UNBOUNDED```
* Not sure if unbounded or infeasible ```GRB.INF_OR_UNBD``` 
  
This can be done by checking ```model.status``` and printing the appropriate status. 
  

In [34]:
if model.status == GRB.OPTIMAL:
    print('The model is feasible and we have found an optimal solution.')
elif model.status == GRB.INF_OR_UNBD:
    print('Model is infeasible or unbounded')
    sys.exit(0)
elif model.status == GRB.INFEASIBLE:
    print('Model is infeasible')
    sys.exit(0)
elif model.status == GRB.UNBOUNDED:
    print('Model is unbounded')
    sys.exit(0)
else:
    print('Optimization ended with status %d' % model.status)
    sys.exit(0)

The model is feasible and we have found an optimal solution.


The current model is optimal. Let us print the optimal objective value using ```model.objVal```.

In [35]:
print(model.objVal)

5.5


Now, let us check the optimal value of $x, y, z$. ```model.getVars()``` gives us the list of all the decision variables. ```k.x``` returns the value, whereas ```k.varName``` returns the name of the variable.

In [36]:
for k in model.getVars():
    print(k.varName ,' = ', k.x)

x  =  1.0
y  =  0.0
z  =  4.5


### How to get the dual variables or shadow prices?
This can be done by typing the constraint name and then use ```.pi```

In [37]:
for k in model.getConstrs():
    print('Shadow price of ', k, '=', k.pi)

Shadow price of  <gurobi.Constr c0> = 0.5
Shadow price of  <gurobi.Constr c1> = 0.5


### What if the model is infeasible. 

In [39]:
model = gp.read('model.lp')
model.Params.outPutFlag = 0
model.optimize()
if model.status == GRB.INFEASIBLE:
    print('Model is infeasible')
    sys.exit(0)
elif model.status == GRB.INF_OR_UNBD:
    print('Model is infeasible or unbounded')
    sys.exit(0)  

Read LP format model from file model.lp
Reading time = 0.00 seconds
: 2 rows, 3 columns, 5 nonzeros


Right now, the solver is not sure if the model is infeasible or unbounded. To clarify this, we set the ```model.Params.DualReductions``` to 0 and then re-optimize.

In [40]:
model.Params.DualReductions  = 0
model.optimize()
if model.status == GRB.INFEASIBLE:
    print('Model is infeasible')

To further explore the infeasibility, we can compute an Irreducible  inconsistent Subsystem (IIS) using ```model.computeIIS()```  and write it as a file. 

In [123]:
model.computeIIS() 
model.write("D:/Down/infeasible_model.lp")


IIS computed: 0 constraints and 2 bounds
IIS runtime: 0.00 seconds


### What if the model is unbounded
We can get the extreme ray using ```UnbdRay```

In [120]:
model.Params.DualReductions  = 0
model.Params.InfUnbdInfo = 1
model.Params.outPutFlag = 0
model.optimize()
for k in model.getVars():
    print(k.varName, ' = ', k.UnbdRay)

x  =  0.0
y  =  0.0
z  =  0.5


## Creating a model and solving it using Gurobi
### Fleet sizing problem
There are three components of above optimization problem:

#### Decisions: 
$x_1$: no. of vans to purchase,
$x_2$: no. of regular buses to purchase, and
$x_3$: no. of articulated buses to purchase
#### Formulation: 

\begin{align*}
& \max_{x_1, x_2, x_3} && 96x_1 + 400x_2 + 900x_3 \\
& \textrm{subject to} && 20x_1 + 120x_2 + 220x_3 \le 2000 \\
&&& x_1 + x_2 + x_3  \le (1-0.1)\times25 \\
&&& 6x_1 + 28x_2 + 56x_3 \ge 450\\
&&& x_1 \ge (x_1 + x_2 + x_3)\times0.3\\
&&& x_2 \ge 10\\
&&& x_3 = 2\\
&&& x_1 \ge 0\\
\end{align*}


In [41]:
from gurobipy import *

m = Model("fleet sizing") # Initializing the model 
x1 = m.addVar(vtype= GRB.CONTINUOUS, lb = 0.0, name="x1") # addVar adds a variable
x2 = m.addVar(vtype= GRB.CONTINUOUS, lb = 0.0, name="x2")
x3 = m.addVar(vtype= GRB.CONTINUOUS, lb = 0.0, name="x3")

budget_constr = m.addConstr(20*x1 + 120*x2 + 220*x3 <= 2000) #addConstr adds a constraint
driver_constr = m.addConstr(x1 + x2 + x3 <= 0.9*25)
ridership_constr = m.addConstr(6*x1 + 28*x2 + 56*x3 >= 450)
van_constr = m.addConstr(x1 >= (x1 +x2 + x3)*0.3)
regbus_constr = m.addConstr(x1 >= 10)
artbus_constr = m.addConstr(x3 == 2)
m.setObjective(96*x1 + 400*x2 + 900*x3, sense = GRB.MAXIMIZE)
m.Params.DualReductions  = 0
m.optimize()


if m.status == 2:
    print('Found optimal solution')
    print(f'x1 = {x1.x}')
    print(f'x2 = {x2.x}')
    print(f'x3 = {x3.x}')
    print(f'overall revenue = {m.objVal}')

x1.vtype = GRB.INTEGER
x2.vtype = GRB.INTEGER
x3.vtype = GRB.INTEGER

m.optimize()

Set parameter DualReductions to value 0
Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (mac64[arm] - Darwin 23.5.0 23F79)

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 6 rows, 3 columns and 14 nonzeros
Model fingerprint: 0x96220b7b
Coefficient statistics:
  Matrix range     [3e-01, 2e+02]
  Objective range  [1e+02, 9e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+00, 2e+03]
Presolve time: 0.00s
Presolved: 3 rows, 2 columns, 6 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    7.9380989e+03   3.060968e+01   0.000000e+00      0s
       2    6.9600000e+03   0.000000e+00   0.000000e+00      0s

Solved in 2 iterations and 0.01 seconds (0.00 work units)
Optimal objective  6.960000000e+03
Found optimal solution
x1 = 10.0
x2 = 10.5
x3 = 2.0
overall revenue = 6960.0
Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (mac64[arm] - Darwin 23.5.0 23F79)

CPU model: Apple M2
T